In [60]:
# 复现2019年东方肝胆的论文
# # 以3年生存期为目标特征，以xgboost、rf、GBDT三个算法建模，计算特征的重要性得分并将其降序排列，分别选取前20的变量

In [61]:
import pandas as pd
import numpy as np

import sys
import re
import os
project_path = os.getcwd()

# load data

In [62]:
df_model_origin =pd.read_excel(project_path+'/data/processed_data/df_3.1.1_术后预后cox数据集.xlsx')
if 'Unnamed: 0' in df_model_origin.columns:
    df_model_origin = df_model_origin.drop(['Unnamed: 0'], axis=1)

In [63]:
df_model=df_model_origin.copy()

In [64]:
df_model.shape

(451, 80)

# 数据清洗

In [65]:
df_model.columns

Index(['id', 'sampling', 'gender', 'age', 'height', 'weight', 'BMI',
       'jaundice', 'emaciation', 'breath_disease', 'cardio_disease', 'nbdd',
       'urinary_disease', 'endocrine_disease', 'biliary_disease',
       'other_disease', 'smoke', 'drinking', 'family_history', 'blood_type',
       'WBC', 'HGB', 'PLT', 'TB', 'DB', 'TP', 'ALB', 'LG', 'AG', 'PAB', 'ALT',
       'AST', 'GT', 'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_CA199',
       'tumor_CA125', 'tumor_size', 'HBsAg', 'HBeAg', 'HBeAb', 'HBcAb',
       'HCVAb', 'LC', 'AJCC_8', 'Gazzaniga_T', 'MSKCC', 'Blumgart_T',
       'Bismuth_C', 'PTCD_ERCP', '手术日期', 'surgery_bleeding', 'surgery_CRCS',
       'surgery_plasm', 'surgery_CP', 'surgery_result', 'gene_mutation',
       'gene_MSI', 'TMB', 'IHC_cdx2', 'IHC_cea', 'IHC_ck5', 'IHC_ck7',
       'IHC_ck19', 'IHC_ck20', 'IHC_muc1', 'IHC_moc31', 'IHC_pd1', 'IHC_pdl1',
       'target_death', 'target_survival_month', 'target_1Y_death',
       'target_1Y_survival', 'target_3Y_death', 'target

## 删除无关特征

In [66]:
df_model=df_model.drop(['id','手术日期','target_death','target_survival_month','target_1Y_death',
                    'target_1Y_survival','target_5Y_death','target_5Y_survival','target_8Y_death','target_8Y_survival'],axis=1)

In [67]:
df_model.shape

(451, 70)

## 删除高线性相关的变量

In [68]:
df_model=df_model.drop(['LG'],axis=1)

## 删除缺失超过50%的数据

In [69]:
# 删除列缺失超过50%
protect_col=['blood_type']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    if df_model[i].isnull().sum()/df_model.shape[0] >= 0.5:
        print(i)
        del df_model[i]

gene_MSI
TMB
IHC_cdx2
IHC_cea
IHC_ck5
IHC_ck7
IHC_ck19
IHC_ck20
IHC_muc1
IHC_moc31
IHC_pd1
IHC_pdl1


In [70]:
# 删除行缺失超过50%
df_model=df_model.reset_index(drop=True)
for i in range(df_model.shape[0]):
    if df_model.loc[i].isnull().sum()/df_model.loc[i].shape[0] >= 0.5:
        print(i)
        df_model.drop([i],axis=0,inplace=True)

194


## 删除分类不平衡特征

In [71]:
# 保护目标变量、tdm数据
protect_col=['sampling']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    if df_model[i].nunique() < 2:
        print(i)
        del df_model[i]
        continue
    if df_model[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_model[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_model.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_model[i]

breath_disease
urinary_disease
family_history


## 过滤异常值

In [72]:
# 过滤异常值，大于正常值超过100倍！
# protect_col=['target_3Y_death','target_3Y_survival']
protect_col=['target_3Y_death','target_3Y_survival','surgery_CRCS','surgery_plasm']
col=[i for i in df_model.columns if i not in protect_col]
for i in col:
    # 测试特征是否存在异常值
#     print(i)
    median_value= max(df_model[i].median(),1)
    max_value = df_model[i].astype(float).max()
    if max_value > 100 * median_value:
        print(i)
    df_model[i]=df_model[i].apply(lambda x: np.nan if pd.isnull(x) else 
                                            np.nan if float(x) > 100*median_value else x)

tumor_AFP


In [73]:
df_model.shape

(450, 54)

In [74]:
df_model.to_excel(project_path+'/data/reimplementation/df_2.4_数据清洗.xlsx')

# 显著性检验

In [75]:
df_model.columns

Index(['sampling', 'gender', 'age', 'height', 'weight', 'BMI', 'jaundice',
       'emaciation', 'cardio_disease', 'nbdd', 'endocrine_disease',
       'biliary_disease', 'other_disease', 'smoke', 'drinking', 'blood_type',
       'WBC', 'HGB', 'PLT', 'TB', 'DB', 'TP', 'ALB', 'AG', 'PAB', 'ALT', 'AST',
       'GT', 'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_CA199', 'tumor_CA125',
       'tumor_size', 'HBsAg', 'HBeAg', 'HBeAb', 'HBcAb', 'HCVAb', 'LC',
       'AJCC_8', 'Gazzaniga_T', 'MSKCC', 'Blumgart_T', 'Bismuth_C',
       'PTCD_ERCP', 'surgery_bleeding', 'surgery_CRCS', 'surgery_plasm',
       'surgery_CP', 'surgery_result', 'gene_mutation', 'target_3Y_death',
       'target_3Y_survival'],
      dtype='object')

In [77]:
# 离散变量和连续变量
discrete_col=['sampling','gender','jaundice','emaciation','cardio_disease','nbdd','endocrine_disease','biliary_disease','other_disease',
              'smoke','drinking','blood_type','HBsAg','HBeAg','HBeAb','HBcAb','HCVAb','LC','AJCC_8','Gazzaniga_T','MSKCC',
              'Blumgart_T','Bismuth_C','PTCD_ERCP','surgery_result','gene_mutation']
continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('target_3Y_death')
continuous_col.remove('target_3Y_survival')

## 分类变量

In [78]:
from scipy.stats import chi2_contingency
feature_list=[]
y_list=[]
t_list=[]
p_list=[]
sig_list=[]
for i in discrete_col:
    print(i)
    result = chi2_contingency(pd.crosstab(df_model['target_3Y_death'],df_model[i]))
    t,p=result[0:2]
    t=round(t,2)
    p=round(p,3)
    feature_list.append(i)
    y_list.append('卡方检验')
    t_list.append(t)
    p_list.append(p)
    if p <=0.05:
        sig='显著'
    else:
        sig='不显著'
    sig_list.append(sig)

sampling
gender
jaundice
emaciation
cardio_disease
nbdd
endocrine_disease
biliary_disease
other_disease
smoke
drinking
blood_type
HBsAg
HBeAg
HBeAb
HBcAb
HCVAb
LC
AJCC_8
Gazzaniga_T
MSKCC
Blumgart_T
Bismuth_C
PTCD_ERCP
surgery_result
gene_mutation


In [79]:
df_discrete_corr_sig=pd.DataFrame(data={'feature':discrete_col,
                                        'method':y_list,
                                        't':t_list,
                                        'p':p_list,
                                        'result':sig_list})

In [80]:
df_discrete_corr_sig

,feature,method,t,p,result
0,sampling,卡方检验,3.30,0.069,不显著
1,gender,卡方检验,0.28,0.599,不显著
2,jaundice,卡方检验,11.93,0.001,显著
3,emaciation,卡方检验,1.04,0.594,不显著
4,cardio_disease,卡方检验,0.00,0.948,不显著
5,nbdd,卡方检验,0.58,0.446,不显著
6,endocrine_disease,卡方检验,1.14,0.285,不显著
7,biliary_disease,卡方检验,0.50,0.477,不显著
8,other_disease,卡方检验,0.96,0.326,不显著
9,smoke,卡方检验,0.09,0.762,不显著


In [81]:
discrete_relate=list(df_discrete_corr_sig[df_discrete_corr_sig.result=='显著']['feature'])

In [82]:
discrete_relate

['jaundice',
 'blood_type',
 'HBcAb',
 'AJCC_8',
 'Gazzaniga_T',
 'surgery_result',
 'gene_mutation']

In [83]:
list(df_discrete_corr_sig[df_discrete_corr_sig.result=='不显著']['feature'])

['sampling',
 'gender',
 'emaciation',
 'cardio_disease',
 'nbdd',
 'endocrine_disease',
 'biliary_disease',
 'other_disease',
 'smoke',
 'drinking',
 'HBsAg',
 'HBeAg',
 'HBeAb',
 'HCVAb',
 'LC',
 'MSKCC',
 'Blumgart_T',
 'Bismuth_C',
 'PTCD_ERCP']

In [84]:
# 舍弃p值大于0.5的变量，保留业务需要的不显著变量
discrete_relate.extend(['sampling','gender', 'emaciation','cardio_disease','endocrine_disease','biliary_disease',
                        'smoke','drinking','HBsAg','HCVAb','LC','PTCD_ERCP','MSKCC','Blumgart_T','Bismuth_C'])

## 连续变量

In [85]:
from scipy.stats import kstest,shapiro
##检验是否正态
def norm_test(data):
    if len(data) > 30:
        norm, p = kstest(data, 'norm')
    else:
        norm, p = shapiro(data)
    #print(t,p)
    if p>=0.05:
        return True
    else:
        return False

In [86]:
import scipy.stats as st
# 连续变量的显著性检验
def test2(data_b, data_p):
    if norm_test(data_b) and norm_test(data_p):
        x = 1
        y = '独立样本T检验'
        t, p = st.ttest_ind(list(data_b),list(data_p), nan_policy='omit')
    else:
        x = 0
        y = 'Mann-Whitney U检验'
        t,p = st.mannwhitneyu(list(data_b),list(data_p))
    return x,y,t,p

In [87]:
def sig_test(df_high,df_low,list1):

    feature_list=[]  # 特征列表
    y_list=[]  # 显著性检验方法
    t_list=[]  # 统计量
    p_list=[]  # p值
    result_list=[]  # 是否显著
    high_mean_list=[]
    low_mean_list=[]

    for i in list1:
        print(i)
        # 高剂量组统计
        df_high_nt=df_high[df_high[i].notnull()]
        data_high=df_high_nt[i]
        high_mean=round(data_high.mean(),2)
        
        df_low_nt=df_low[df_low[i].notnull()]
        data_low=df_low_nt[i]
        low_mean=round(data_low.mean(),2)

        # 计算高低剂量组显著性差异
        if data_high.shape[0] >= 10 and data_low.shape[0]>=10:
            # 连续变量检验
            x,y,t,p = test2(data_high, data_low)
            t=round(t,2)
            p=round(p,3)
            if p <=0.05:
                sig='显著'
            else:
                sig='不显著'
            # 显著性 
            feature_list.append(i)
            y_list.append(y)
            t_list.append(t)
            p_list.append(p)
            result_list.append(sig)
            high_mean_list.append(high_mean)
            low_mean_list.append(low_mean)

    df_result=pd.DataFrame({'特征':feature_list,
                            '高剂量均值':high_mean_list,
                            '低剂量均值':low_mean_list,
                            '检验指标':y_list,
                            't值':t_list,
                            'p值':p_list,
                            '显著性结果':result_list})
    return df_result

In [88]:
# 提取甲氨蝶呤不同标签
df_MTX_high = df_model[df_model['target_3Y_death']==1]
df_MTX_low = df_model[df_model['target_3Y_death']==0]
# 甲氨蝶呤高低剂量组PSM控制变量统计和显著性检验
df_continuous_sig = sig_test(df_MTX_high,df_MTX_low,continuous_col)
df_continuous_sig = df_continuous_sig.reset_index(drop=True)

age
height
weight
BMI
WBC
HGB
PLT
TB
DB
TP
ALB
AG
PAB
ALT
AST
GT
ALP
tumor_AFP
tumor_CEA
tumor_CA199
tumor_CA125
tumor_size
surgery_bleeding
surgery_CRCS
surgery_plasm
surgery_CP


In [89]:
df_continuous_sig

,特征,高剂量均值,低剂量均值,检验指标,t值,p值,显著性结果
0,age,57.57,59.04,Mann-Whitney U检验,16341.5,0.101,不显著
1,height,164.97,167.75,Mann-Whitney U检验,2594.5,0.029,显著
2,weight,61.58,65.55,Mann-Whitney U检验,2637.5,0.036,显著
3,BMI,22.56,23.30,Mann-Whitney U检验,2939.0,0.143,不显著
4,WBC,6.39,6.29,Mann-Whitney U检验,10028.5,0.303,不显著
5,HGB,123.28,125.41,Mann-Whitney U检验,9373.5,0.115,不显著
6,PLT,255.61,222.89,Mann-Whitney U检验,7779.5,0.005,显著
7,TB,186.43,155.16,Mann-Whitney U检验,14439.5,0.004,显著
8,DB,145.83,138.52,Mann-Whitney U检验,9212.0,0.144,不显著
9,TP,64.78,65.77,Mann-Whitney U检验,12602.5,0.181,不显著


In [90]:
continuous_relate = list(df_continuous_sig[df_continuous_sig['显著性结果']=='显著']['特征'])

In [91]:
continuous_relate

['height',
 'weight',
 'PLT',
 'TB',
 'tumor_CA199',
 'tumor_CA125',
 'surgery_bleeding']

In [92]:
list(df_continuous_sig[df_continuous_sig['显著性结果']=='不显著']['特征'])

['age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'TP',
 'ALB',
 'AG',
 'PAB',
 'ALT',
 'AST',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size',
 'surgery_CRCS',
 'surgery_plasm',
 'surgery_CP']

In [93]:
# 舍弃p值大于0.5的变量，保留医学相关的变量：
continuous_relate.extend(['age','BMI','WBC','HGB','DB','ALB','GT','ALP','tumor_AFP','tumor_CEA','tumor_size','surgery_plasm'])

In [94]:
continuous_relate

['height',
 'weight',
 'PLT',
 'TB',
 'tumor_CA199',
 'tumor_CA125',
 'surgery_bleeding',
 'age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'ALB',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size',
 'surgery_plasm']

## 提取显著特征

In [95]:
# 提取相关性特征
feature_relate=discrete_relate + continuous_relate
# 加入目标变量
feature_relate.insert(0,'target_3Y_survival')
feature_relate.insert(0,'target_3Y_death')

df_model_relate=df_model[feature_relate]

In [96]:
df_model_relate.shape

(450, 43)

In [97]:
feature_relate

['target_3Y_death',
 'target_3Y_survival',
 'jaundice',
 'blood_type',
 'HBcAb',
 'AJCC_8',
 'Gazzaniga_T',
 'surgery_result',
 'gene_mutation',
 'sampling',
 'gender',
 'emaciation',
 'cardio_disease',
 'endocrine_disease',
 'biliary_disease',
 'smoke',
 'drinking',
 'HBsAg',
 'HCVAb',
 'LC',
 'PTCD_ERCP',
 'MSKCC',
 'Blumgart_T',
 'Bismuth_C',
 'height',
 'weight',
 'PLT',
 'TB',
 'tumor_CA199',
 'tumor_CA125',
 'surgery_bleeding',
 'age',
 'BMI',
 'WBC',
 'HGB',
 'DB',
 'ALB',
 'GT',
 'ALP',
 'tumor_AFP',
 'tumor_CEA',
 'tumor_size',
 'surgery_plasm']

In [98]:
df_model_relate.to_excel(project_path+'/data/reimplementation/df_3.3_显著性检验.xlsx')

# split data

In [99]:
df_model_split=df_model_relate.copy()

In [100]:
df_model_split.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'blood_type',
       'HBcAb', 'AJCC_8', 'Gazzaniga_T', 'surgery_result', 'gene_mutation',
       'sampling', 'gender', 'emaciation', 'cardio_disease',
       'endocrine_disease', 'biliary_disease', 'smoke', 'drinking', 'HBsAg',
       'HCVAb', 'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C',
       'height', 'weight', 'PLT', 'TB', 'tumor_CA199', 'tumor_CA125',
       'surgery_bleeding', 'age', 'BMI', 'WBC', 'HGB', 'DB', 'ALB', 'GT',
       'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_size', 'surgery_plasm'],
      dtype='object')

In [102]:
# 离散变量和连续变量
discrete_col=['jaundice', 'blood_type','HBcAb', 'AJCC_8', 'Gazzaniga_T', 'surgery_result', 'gene_mutation',
               'sampling', 'gender', 'emaciation', 'cardio_disease','endocrine_disease', 'biliary_disease',
              'smoke', 'drinking', 'HBsAg','HCVAb', 'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C',]
continuous_col=[x for x in df_model_split.columns if x not in discrete_col]
continuous_col.remove('target_3Y_death')
continuous_col.remove('target_3Y_survival')

## 数据归一化

In [103]:
# 防止不同维特征数据差距过大，影响建模效果
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_model_split[continuous_col]=ss.fit_transform(df_model_split[continuous_col])

## 插补数据

In [104]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df):
    df = df.reset_index(drop=True)
    
    # 提取存在缺失值的列名
    missing_list = []
    # 设定保护列
    protect_col=['target_3Y_death', 'target_3Y_survival','sampling']
    for i in df.columns:
        if i in protect_col:
            continue
        try:
            if df[i].isnull().sum()>0:
                missing_list.append(i)
        except:
            if df[i].isnull().sum()[0]>0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)

        rfr = RandomForestRegressor(n_estimators=300,
                                    random_state=3)
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

In [105]:
# 插补建模数据
df_model_cb=missing_value_interpolation(df_model_split)
# df_model_cb=df_model

## 划分数据集

In [106]:
# 分类随机数种子
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
y = df_model_cb['target_3Y_death']

tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)

In [107]:
# 进行过采样
from imblearn.over_sampling import SMOTE,ADASYN 
from imblearn.combine import SMOTETomek
sm = SMOTE(random_state=0)
# sm=ADASYN(random_state=0)

tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)

# models

In [108]:
'''
分别训练xgboost,rf,GBDT三个模型，并提起重要性排名前20的特征
'''

'\n分别训练xgboost,rf,GBDT三个模型，并提起重要性排名前20的特征\n'

## XGBoost

In [109]:
import xgboost
xgb_model=xgboost.XGBClassifier(max_depth=4,
                        learning_rate=0.01,
                        n_estimators=500,
                        min_child_weight=0.8,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=6,
                        subsample=0.8,
                        colsample_bytree=0.6,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)

xgb_model.fit(tran_x_sm,tran_y_sm)

importance = xgb_model.feature_importances_
print(importance)

[0.03181199 0.01902641 0.0280488  0.03520014 0.0230205  0.03555881
 0.00917477 0.02110408 0.02058736 0.04020663 0.01480667 0.01939563
 0.02546807 0.031596   0.01603773 0.0080425  0.04177326 0.03172116
 0.02419323 0.02793033 0.01009269 0.03605413 0.05509132 0.04035041
 0.01707709 0.01148689 0.02096331 0.01921927 0.01551474 0.07875545
 0.01758858 0.0315752  0.01668495 0.01231433 0.02118156 0.02059075
 0.01374187 0.01460844 0.0165038  0.02590117]


In [110]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [111]:
df_importance

,特征,重要性评分
0,BMI,0.079
1,weight,0.055
2,LC,0.042
3,PLT,0.040
4,cardio_disease,0.040
5,surgery_result,0.036
6,height,0.036
7,AJCC_8,0.035
8,jaundice,0.032
9,HGB,0.032


In [112]:
# 取前20个变量
col_xgb_20=df_importance.loc[:19,'特征'].to_list()

In [113]:
col_xgb_20

['BMI',
 'weight',
 'LC',
 'PLT',
 'cardio_disease',
 'surgery_result',
 'height',
 'AJCC_8',
 'jaundice',
 'HGB',
 'PTCD_ERCP',
 'drinking',
 'HBcAb',
 'Blumgart_T',
 'surgery_plasm',
 'smoke',
 'MSKCC',
 'Gazzaniga_T',
 'tumor_CA125',
 'gender']

## RF

In [114]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
# 进行参数的搜索组合
grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x_sm, tran_y_sm)
rf_model = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                            max_depth=8,
                            random_state=3)
rf_model.fit(tran_x_sm, tran_y_sm)

importance = rf_model.feature_importances_
print(importance)

[0.02280943 0.04156124 0.00829138 0.02601147 0.01437392 0.03871985
 0.00222983 0.00382665 0.01393732 0.00962586 0.00408092 0.0091972
 0.00980939 0.0189579  0.00662847 0.00130855 0.00916904 0.01536364
 0.0053214  0.00671829 0.01192693 0.06278662 0.09314706 0.06046459
 0.02599249 0.0187143  0.03301184 0.02465981 0.01810587 0.1265724
 0.02453661 0.04949208 0.02304437 0.02022157 0.02971787 0.02809836
 0.0214031  0.01516062 0.01977108 0.02523067]


In [115]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [116]:
df_importance

,特征,重要性评分
0,BMI,0.127
1,weight,0.093
2,height,0.063
3,PLT,0.060
4,HGB,0.049
5,blood_type,0.042
6,surgery_result,0.039
7,tumor_CA125,0.033
8,GT,0.030
9,ALP,0.028


In [117]:
# 取前20个变量
col_rf_20=df_importance.loc[:19,'特征'].to_list()

In [118]:
col_rf_20

['BMI',
 'weight',
 'height',
 'PLT',
 'HGB',
 'blood_type',
 'surgery_result',
 'tumor_CA125',
 'GT',
 'ALP',
 'AJCC_8',
 'TB',
 'surgery_plasm',
 'WBC',
 'surgery_bleeding',
 'DB',
 'jaundice',
 'tumor_AFP',
 'tumor_size',
 'ALB']

## GBDT

In [119]:
# GBDT
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
gbdt_model = GradientBoostingClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=8,
                            subsample=0.4,
                            random_state=3)
gbdt_model.fit(tran_x_sm, tran_y_sm)

importance = gbdt_model.feature_importances_
print(importance)

[0.00754429 0.0469273  0.01276588 0.01775737 0.0136081  0.0207538
 0.00201159 0.00068379 0.01450367 0.01015251 0.00679988 0.01268433
 0.00549206 0.0071727  0.00493811 0.01336529 0.02068087 0.01879923
 0.0014247  0.0009145  0.01469837 0.03041213 0.05481928 0.06995983
 0.01235762 0.02269088 0.03534739 0.01880169 0.02272085 0.1781665
 0.0216087  0.07892678 0.01370287 0.01336861 0.04676374 0.03042213
 0.03113872 0.01988916 0.0256533  0.01957149]


In [120]:
df_importance= pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [121]:
df_importance

,特征,重要性评分
0,BMI,0.178
1,HGB,0.079
2,PLT,0.070
3,weight,0.055
4,blood_type,0.047
5,GT,0.047
6,tumor_CA125,0.035
7,tumor_AFP,0.031
8,ALP,0.030
9,height,0.030


In [122]:
# 取前20个变量
col_gbdt_20=df_importance.loc[:19,'特征'].to_list()

In [123]:
col_gbdt_20

['BMI',
 'HGB',
 'PLT',
 'weight',
 'blood_type',
 'GT',
 'tumor_CA125',
 'tumor_AFP',
 'ALP',
 'height',
 'tumor_size',
 'tumor_CA199',
 'age',
 'WBC',
 'surgery_result',
 'LC',
 'tumor_CEA',
 'surgery_plasm',
 'surgery_bleeding',
 'PTCD_ERCP']

## 取重要变量交集

In [124]:
df_col_importance=pd.DataFrame(data={'xgb_col':col_xgb_20,
                                    'rf_col':col_rf_20,
                                    'gbdt_col':col_gbdt_20})

In [125]:
df_col_importance

,xgb_col,rf_col,gbdt_col
0,BMI,BMI,BMI
1,weight,weight,HGB
2,LC,height,PLT
3,PLT,PLT,weight
4,cardio_disease,HGB,blood_type
5,surgery_result,blood_type,GT
6,height,surgery_result,tumor_CA125
7,AJCC_8,tumor_CA125,tumor_AFP
8,jaundice,GT,ALP
9,HGB,ALP,height


In [126]:
df_col_importance.to_excel(project_path+'/data/reimplementation/df_5.4_top20重要变量.xlsx')

In [127]:
# col_xgb_20, col_rf_20, col_gbdt_20，得到10个交集变量
col_importance_merge=[i for i in col_xgb_20 if i in col_rf_20]
col_importance_merge=[i for i in col_gbdt_20 if i in col_importance_merge]

In [128]:
col_importance_merge

['BMI',
 'HGB',
 'PLT',
 'weight',
 'tumor_CA125',
 'height',
 'surgery_result',
 'surgery_plasm']

In [129]:
## 插入目标变量
col_importance_merge.insert(0,'sampling')
col_importance_merge.extend(['target_3Y_death', 'target_3Y_survival'])

In [130]:
col_importance_merge

['sampling',
 'BMI',
 'HGB',
 'PLT',
 'weight',
 'tumor_CA125',
 'height',
 'surgery_result',
 'surgery_plasm',
 'target_3Y_death',
 'target_3Y_survival']

In [131]:
df_model_merge=df_model_relate[col_importance_merge]

In [132]:
df_model_merge.to_excel(project_path+'/data/reimplementation/df_5.4_取重要变量交集.xlsx')

# 计算C_index

## 所有变量

In [143]:
df_model_cox_all=df_model_relate[df_model_relate.target_3Y_survival.notnull() & df_model_relate.sampling.notnull()]

In [144]:
df_model_cox_all.shape

(362, 43)

In [145]:
df_model_cox_all.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'blood_type',
       'HBcAb', 'AJCC_8', 'Gazzaniga_T', 'surgery_result', 'gene_mutation',
       'sampling', 'gender', 'emaciation', 'cardio_disease',
       'endocrine_disease', 'biliary_disease', 'smoke', 'drinking', 'HBsAg',
       'HCVAb', 'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T', 'Bismuth_C',
       'height', 'weight', 'PLT', 'TB', 'tumor_CA199', 'tumor_CA125',
       'surgery_bleeding', 'age', 'BMI', 'WBC', 'HGB', 'DB', 'ALB', 'GT',
       'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_size', 'surgery_plasm'],
      dtype='object')

### 查看缺失率

In [146]:
# 删除列缺失超过50%
for i in df_model_cox_all.columns:
    percent_col= df_model_cox_all[i].isnull().sum()/df_model_cox_all.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%
每列缺失率: jaundice 0.00%
每列缺失率: blood_type 64.64%
每列缺失率: HBcAb 12.98%
每列缺失率: AJCC_8 5.80%
每列缺失率: Gazzaniga_T 6.91%
每列缺失率: surgery_result 9.67%
每列缺失率: gene_mutation 0.00%
每列缺失率: sampling 0.00%
每列缺失率: gender 0.28%
每列缺失率: emaciation 14.36%
每列缺失率: cardio_disease 0.28%
每列缺失率: endocrine_disease 0.55%
每列缺失率: biliary_disease 0.55%
每列缺失率: smoke 0.00%
每列缺失率: drinking 4.14%
每列缺失率: HBsAg 8.56%
每列缺失率: HCVAb 13.26%
每列缺失率: LC 22.10%
每列缺失率: PTCD_ERCP 5.25%
每列缺失率: MSKCC 6.91%
每列缺失率: Blumgart_T 6.91%
每列缺失率: Bismuth_C 6.35%
每列缺失率: height 38.67%
每列缺失率: weight 38.67%
每列缺失率: PLT 16.02%
每列缺失率: TB 1.10%
每列缺失率: tumor_CA199 1.38%
每列缺失率: tumor_CA125 33.15%
每列缺失率: surgery_bleeding 8.84%
每列缺失率: age 0.28%
每列缺失率: BMI 38.67%
每列缺失率: WBC 14.36%
每列缺失率: HGB 15.19%
每列缺失率: DB 14.09%
每列缺失率: ALB 3.87%
每列缺失率: GT 30.39%
每列缺失率: ALP 32.87%
每列缺失率: tumor_AFP 4.70%
每列缺失率: tumor_CEA 2.49%
每列缺失率: tumor_size 22.65%
每列缺失率: surgery_plasm 9.67%


In [147]:
# 删除数据缺失超过30%的重要变量，否则变量数超过数据量，cox无法运行
missing_col=['blood_type','height','weight','tumor_CA125','BMI','GT','ALP']
df_model_cox_all=df_model_cox_all.drop(missing_col,axis=1)
# 删除损害收敛性的变量
df_model_cox_all=df_model_cox_all.drop(['HCVAb'],axis=1)

### dataset for cox-all

In [148]:
# 计算3年生存期分期的c_index_all
for i in df_model_cox_all.columns:
    df_model_cox_all = df_model_cox_all[df_model_cox_all[i].notnull()]

In [149]:
df_model_cox_all.sampling.value_counts()

1    128
0      1
Name: sampling, dtype: int64

In [150]:
df_model_cox_all.shape

(129, 35)

In [151]:
df_model_cox_all.to_excel(project_path+'/data/reimplementation/df_6.1.2_dataset_cox_all.xlsx')

### c_index_all

In [152]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

cph = CoxPHFitter()
cph.fit(df_model_cox_all, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 2.73420, step_size = 0.9000, log_lik = -394.63353, newton_decrement = 58.13629, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.81957, step_size = 0.9000, log_lik = -353.33897, newton_decrement = 9.51915, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.15463, step_size = 0.9000, log_lik = -343.90408, newton_decrement = 0.46884, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.09016, step_size = 1.0000, log_lik = -343.39324, newton_decrement = 0.02870, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.08806, step_size = 1.0000, log_lik = -343.35845, newton_decrement = 0.00836, seconds_since_start = 0.0
Iteration 6: norm_delta = 0.08806, step_size = 1.0000, log_lik = -343.34788, newton_decrement = 0.00307, seconds_since_start = 0.0
Iteration 7: norm_delta = 0.08805, step_size = 1.0000, log_lik = -343.34400, newton_decrement = 0.00113, seconds_since_start = 0.0
Iteration 8: norm_delta = 0.08805, step_size = 1.0000, log_lik = -343.34257, newto

## ML筛选的变量

In [153]:
df_model_cox_ML=df_model_merge[df_model_merge.target_3Y_survival.notnull() & df_model_merge.sampling.notnull()]

In [154]:
df_model_cox_ML.columns

Index(['sampling', 'BMI', 'HGB', 'PLT', 'weight', 'tumor_CA125', 'height',
       'surgery_result', 'surgery_plasm', 'target_3Y_death',
       'target_3Y_survival'],
      dtype='object')

In [155]:
df_model_cox_ML.shape

(362, 11)

### 查看缺失率

In [156]:
# 删除列缺失超过50%
for i in df_model_cox_ML.columns:
    percent_col= df_model_cox_ML[i].isnull().sum()/df_model_cox_ML.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: sampling 0.00%
每列缺失率: BMI 38.67%
每列缺失率: HGB 15.19%
每列缺失率: PLT 16.02%
每列缺失率: weight 38.67%
每列缺失率: tumor_CA125 33.15%
每列缺失率: height 38.67%
每列缺失率: surgery_result 9.67%
每列缺失率: surgery_plasm 9.67%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


In [158]:
# 删除数据缺失超过30%的重要变量，否则变量数超过数据量，cox无法运行
missing_col=['height','weight','BMI','tumor_CA125']
df_model_cox_ML=df_model_cox_ML.drop(missing_col,axis=1)
# 删除损害收敛性的变量

### dataset for cox-ML

In [159]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_ML.columns:
    df_model_cox_ML = df_model_cox_ML[df_model_cox_ML[i].notnull()]

In [160]:
df_model_cox_ML.sampling.value_counts()

1    273
0      1
Name: sampling, dtype: int64

In [161]:
df_model_cox_ML.shape

(274, 7)

In [162]:
df_model_cox_ML.to_excel(project_path+'/data/reimplementation/df_6.2.2_dataset_cox_ML.xlsx')

### c_index_ML

In [163]:
cph = CoxPHFitter()
cph.fit(df_model_cox_ML, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.38724, step_size = 0.9000, log_lik = -1095.37011, newton_decrement = 85.12252, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.33150, step_size = 0.9000, log_lik = -1042.82694, newton_decrement = 10.29848, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.03714, step_size = 0.9000, log_lik = -1032.64207, newton_decrement = 0.11789, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00021, step_size = 1.0000, log_lik = -1032.52372, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1032.52372, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.726634043040293


## SPSS：backward筛选的变量

In [164]:
col_backward=['jaundice','AJCC_8','surgery_result','gene_mutation','cardio_disease','smoke','LC','Bismuth_C','tumor_CA199',
            'DB','tumor_AFP','tumor_size']
col_backward.insert(0,'sampling')
col_backward.extend(['target_3Y_death','target_3Y_survival'])

In [165]:
df_model_cox_back=df_model_relate[col_backward]
df_model_cox_back=df_model_cox_back[df_model_cox_back.target_3Y_survival.notnull()]

### 查看缺失率

In [167]:
# 删除列缺失超过50%
for i in df_model_cox_back.columns:
    percent_col= df_model_cox_back[i].isnull().sum()/df_model_cox_back.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: sampling 0.00%
每列缺失率: jaundice 0.00%
每列缺失率: AJCC_8 5.80%
每列缺失率: surgery_result 9.67%
每列缺失率: gene_mutation 0.00%
每列缺失率: cardio_disease 0.28%
每列缺失率: smoke 0.00%
每列缺失率: LC 22.10%
每列缺失率: Bismuth_C 6.35%
每列缺失率: tumor_CA199 1.38%
每列缺失率: DB 14.09%
每列缺失率: tumor_AFP 4.70%
每列缺失率: tumor_size 22.65%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


### dataset for cox-back

In [170]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_back.columns:
    df_model_cox_back = df_model_cox_back[df_model_cox_back[i].notnull()]

In [171]:
df_model_cox_back.shape

(168, 15)

In [172]:
df_model_cox_back.sampling.value_counts()

1    167
0      1
Name: sampling, dtype: int64

In [174]:
df_model_cox_back.to_excel(project_path+'/data/reimplementation/df_6.3.2_dataset_cox_back.xlsx')

### c_index_back

In [175]:
cph = CoxPHFitter()
cph.fit(df_model_cox_back, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.29325, step_size = 0.9000, log_lik = -563.35479, newton_decrement = 60.08919, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.37728, step_size = 0.9000, log_lik = -519.78073, newton_decrement = 8.27734, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.04973, step_size = 0.9000, log_lik = -511.44462, newton_decrement = 0.18777, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00111, step_size = 1.0000, log_lik = -511.25402, newton_decrement = 0.00011, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -511.25391, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.7524907821448183


## ML+SPSS筛选的变量

In [219]:
# 人工设定3个
col_ML_back=['PLT','AJCC_8','surgery_result','surgery_plasm']
col_ML_back.insert(0,'sampling')
col_ML_back.extend(['target_3Y_death','target_3Y_survival'])

In [220]:
df_model_cox_ML_back=df_model_relate[col_ML_back]
df_model_cox_ML_back=df_model_cox_ML_back[df_model_cox_ML_back.target_3Y_survival.notnull() ]

### 查看缺失率

In [221]:
# 删除列缺失超过50%
for i in df_model_cox_ML_back.columns:
    percent_col= df_model_cox_ML_back[i].isnull().sum()/df_model_cox_ML_back.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: sampling 0.00%
每列缺失率: PLT 16.02%
每列缺失率: AJCC_8 5.80%
每列缺失率: surgery_result 9.67%
每列缺失率: surgery_plasm 9.67%
每列缺失率: target_3Y_death 0.00%
每列缺失率: target_3Y_survival 0.00%


### dataset for cox-ML-back

In [222]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算3年生存期分期的c_index_all
for i in df_model_cox_ML_back.columns:
    df_model_cox_ML_back = df_model_cox_ML_back[df_model_cox_ML_back[i].notnull()]

In [223]:
df_model_cox_ML_back.shape

(275, 7)

In [224]:
df_model_cox_ML_back.sampling.value_counts()

1    274
0      1
Name: sampling, dtype: int64

In [225]:
df_model_cox_ML_back.to_excel(project_path+'/data/reimplementation/df_6.4.2_dataset_cox_ML_back.xlsx')

### c_index_ML_back

In [226]:
cph = CoxPHFitter()
cph.fit(df_model_cox_ML_back, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
c_index_all = cph.concordance_index_
print(c_index_all)

Iteration 1: norm_delta = 1.24868, step_size = 0.9000, log_lik = -1100.98688, newton_decrement = 93.08482, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.35976, step_size = 0.9000, log_lik = -1036.90372, newton_decrement = 9.64815, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.03699, step_size = 0.9000, log_lik = -1027.36126, newton_decrement = 0.10799, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00018, step_size = 1.0000, log_lik = -1027.25290, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1027.25290, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.7348613951374687


In [227]:
cph.print_summary(col=['coef'])

model,lifelines.CoxPHFitter
duration col,'target_3Y_survival'
event col,'target_3Y_death'
cluster col,'sampling'
robust variance,True
baseline estimation,breslow
number of observations,275
number of events observed,219
partial log-likelihood,-1027.25
time fit was run,2022-03-10 11:52:31 UTC
col,[coef]


# 计算BS

In [470]:
# 计算3年生存期的BS
from sksurv.metrics import brier_score
from sksurv.datasets import load_gbsg2
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import brier_score
from sksurv.preprocessing import OneHotEncoder

ContextualVersionConflict: (scikit-learn 0.24.2 (c:\users\administrator\appdata\roaming\python\python36\site-packages), Requirement.parse('scikit-learn<0.24,>=0.22.0'), {'scikit-survival'})

## 所有变量

In [184]:
df_model_cox_all=pd.read_excel(project_path+'/data/reimplementation/df_6.1.2_dataset_cox_all.xlsx')
if 'Unnamed: 0' in df_model_cox_all.columns:
    df_model_cox_all = df_model_cox_all.drop(['Unnamed: 0'], axis=1)

In [185]:
# 转换成boolean值
df_model_cox_all['target_death_binary']=df_model_cox_all.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_all.target_death_binary,df_model_cox_all.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [186]:
df_model_cox_all.columns

Index(['target_3Y_death', 'target_3Y_survival', 'jaundice', 'HBcAb', 'AJCC_8',
       'Gazzaniga_T', 'surgery_result', 'gene_mutation', 'sampling', 'gender',
       'emaciation', 'cardio_disease', 'endocrine_disease', 'biliary_disease',
       'smoke', 'drinking', 'HBsAg', 'LC', 'PTCD_ERCP', 'MSKCC', 'Blumgart_T',
       'Bismuth_C', 'PLT', 'TB', 'tumor_CA199', 'surgery_bleeding', 'age',
       'WBC', 'HGB', 'DB', 'ALB', 'tumor_AFP', 'tumor_CEA', 'tumor_size',
       'surgery_plasm', 'target_death_binary'],
      dtype='object')

In [472]:
# 术前3年BS_all
bs_x_all = df_model_cox_all.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_all = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_all, bs_y)
survs_all = est_all.predict_survival_function(bs_x_all)
preds_all = [fn(10) for fn in survs_all]
times, bs_3Y_all = brier_score(bs_y, bs_y, preds_all, 10)
print(bs_3Y_all)

KeyError: "['target_3Y_death' 'target_3Y_survival' 'sampling'] not found in axis"

## ML筛选的变量

In [188]:
df_model_cox_ML=pd.read_excel(project_path+'/data/reimplementation/df_6.2.2_dataset_cox_ML.xlsx')
if 'Unnamed: 0' in df_model_cox_ML.columns:
    df_model_cox_ML = df_model_cox_ML.drop(['Unnamed: 0'], axis=1)

In [189]:
df_model_cox_ML.shape

(274, 7)

In [ ]:
# 转换成布尔值
df_model_cox_ML['target_death_binary']=df_model_cox_ML.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_ML.target_death_binary,df_model_cox_ML.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术前3年BS_ML
bs_x_ML = df_model_cox_ML.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_ML = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ML, bs_y)
survs_ML = est_ML.predict_survival_function(bs_x_ML)
preds_ML = [fn(10) for fn in survs_ML]
times, bs_3Y_ML = brier_score(bs_y, bs_y, preds_ML, 10)
print(bs_3Y_ML)

## SPSS:backward筛选的变量

In [190]:
df_model_cox_backward=pd.read_excel(project_path+'/data/reimplementation/df_6.3.2_dataset_cox_back.xlsx')
if 'Unnamed: 0' in df_model_cox_backward.columns:
    df_model_cox_backward = df_model_cox_backward.drop(['Unnamed: 0'], axis=1)

In [191]:
df_model_cox_backward.shape

(168, 15)

In [ ]:
# 转换成布尔值
df_model_cox_backward['target_death_binary']=df_model_cox_backward.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_backward.target_death_binary,df_model_cox_backward.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术前3年BS_backward
bs_x_backward = df_model_cox_backward.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_backward = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_backward, bs_y)
survs_backward = est_backward.predict_survival_function(bs_x_backward)
preds_backward = [fn(10) for fn in survs_backward]
times, bs_3Y_backward = brier_score(bs_y, bs_y, preds_backward, 10)
print(bs_3Y_backward)

## ML+SPSS筛选的变量

In [199]:
df_model_cox_ML_back=pd.read_excel(project_path+'/data/reimplementation/df_6.4.2_dataset_cox_ML_back.xlsx')
if 'Unnamed: 0' in df_model_cox_ML_back.columns:
    df_model_cox_ML_back = df_model_cox_ML_back.drop(['Unnamed: 0'], axis=1)

In [200]:
df_model_cox_ML_back.shape

(276, 6)

In [197]:
# 转换成布尔值
df_model_cox_ML_back['target_death_binary']=df_model_cox_ML_back.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_cox_ML_back.target_death_binary,df_model_cox_ML_back.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [198]:
# 术前3年BS_ML_back
bs_x_ML_back = df_model_cox_ML_back.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
est_ML_back = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ML_back, bs_y)
survs_ML_back = est_ML_back.predict_survival_function(bs_x_ML_back)
preds_ML_back = [fn(10) for fn in survs_ML_back]
times, bs_3Y_ML_back = brier_score(bs_y, bs_y, preds_ML_back, 10)
print(bs_3Y_ML_back)

NameError: name 'CoxPHSurvivalAnalysis' is not defined

## 保存BS值

In [474]:
df_bs = pd.DataFrame(data={'bs_3Y_all':bs_3Y_all,
                          'bs_3Y_ML':bs_3Y_ML,
                          'bs_3Y_backward':bs_3Y_backward,
                          'bs_3Y_ML_back':bs_3Y_ML_back})
df_bs.to_excel(project_path+'/data/implementation/df_bs.xlsx')

NameError: name 'bs_3Y_all' is not defined

# 构建风险评分模型

# 计算风险评分

In [245]:
# 根据ML+SPSS筛选的变量构建风险模型:'PLT','surgery_result','surgery_plasm'
import math
# 计算风险评分
df_model_risk_before=df_model_relate.reset_index(drop=True)
# 缺失项补充为0
discrete_col=['surgery_result','AJCC_8']
df_model_risk_before[discrete_col]=df_model_risk_before[discrete_col].fillna(0)
cotinuous_col=['PLT','surgery_plasm']
df_model_risk_before[cotinuous_col]=df_model_risk_before[cotinuous_col].fillna(1)
for i in range(df_model_risk_before.shape[0]):
    surgery_result=float(df_model_risk_before.loc[i,'surgery_result'])
    plt_value=float(df_model_risk_before.loc[i,'PLT'])
    surgery_plasm=float(df_model_risk_before.loc[i,'surgery_plasm'])
    ajcc=float(df_model_risk_before.loc[i,'AJCC_8'])
#     db_value=float(df_model_risk_before.loc[i,'DB'])
#     plt=float(df_model_risk_before.loc[i,'PLT'])
    # 术前1年生存期risk_score计算
    df_model_risk_before.loc[i,'risk_score']=math.log(plt_value)*0.002+ajcc*0.58+0.59*surgery_result+0.0004*surgery_plasm
#     df_model_risk_before.loc[i,'risk_score']=math.log(plt)*0.002+0.41*jaundice_value+0.41*gazz_value

In [246]:
df_model_risk_before.risk_score.describe()

count    450.000000
mean       2.414852
std        1.101622
min        0.000400
25%        1.761300
50%        2.340675
75%        2.915199
max        5.011693
Name: risk_score, dtype: float64

## 分期

In [247]:
# 术前1年生存期风险评分分期
df_model_risk_before['staging']=df_model_risk_before.risk_score.apply(lambda x: 1 if x<=1.5 else
                                                                     2 if 1.5<x<=2.5 else
                                                                     3 if 2.5<x<=3 else
                                                                     4 if x>3 else np.nan)

In [248]:
df_model_risk_before.staging.value_counts()

2    251
4     99
3     64
1     36
Name: staging, dtype: int64

In [249]:
df_model_risk_before.to_excel(project_path+'/data/reimplementation/df_8.1_术后分期.xlsx')

## 计算C_index

### c_index_us

In [250]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# 计算1年生存期分期的c_index_us
df_C_index_us = df_model_risk_before[['staging','target_3Y_death','target_3Y_survival']]
df_C_index_us = df_C_index_us[df_C_index_us.target_3Y_survival.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_us, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_us = cph.concordance_index_
print(c_index_us)

Iteration 1: norm_delta = 0.62534, step_size = 0.9000, log_lik = -1397.02600, newton_decrement = 35.93323, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.02478, step_size = 0.9000, log_lik = -1364.89529, newton_decrement = 0.07274, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00253, step_size = 0.9000, log_lik = -1364.82321, newton_decrement = 0.00075, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1364.82246, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6375589733108511


### c_index_ajcc

In [251]:
# 计算1年生存期分期的c_index_ajcc
df_C_index_ajcc = df_model_risk_before[['AJCC_8','target_3Y_death','target_3Y_survival']]
df_C_index_ajcc = df_C_index_ajcc[df_C_index_ajcc.target_3Y_survival.notnull() & df_C_index_ajcc.AJCC_8.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_ajcc, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_ajcc = cph.concordance_index_
print(c_index_ajcc)

Iteration 1: norm_delta = 0.34658, step_size = 0.9000, log_lik = -1397.02600, newton_decrement = 11.84830, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.09256, step_size = 0.9000, log_lik = -1384.48964, newton_decrement = 0.65963, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.01199, step_size = 0.9000, log_lik = -1383.82733, newton_decrement = 0.01052, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00004, step_size = 1.0000, log_lik = -1383.81679, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 5: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1383.81679, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 5 iterations.
0.6192921499497556


### c_index_gazz

In [252]:
# 计算1年生存期分期的c_index_gazz
df_C_index_gazz = df_model_risk_before[['Gazzaniga_T','target_3Y_death','target_3Y_survival']]
df_C_index_gazz = df_C_index_gazz[df_C_index_gazz.target_3Y_survival.notnull() & df_C_index_gazz.Gazzaniga_T.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_gazz, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_gazz = cph.concordance_index_
print(c_index_gazz)

Iteration 1: norm_delta = 0.49225, step_size = 0.9000, log_lik = -1273.65809, newton_decrement = 21.87271, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.03542, step_size = 0.9000, log_lik = -1254.72405, newton_decrement = 0.16068, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00385, step_size = 0.9000, log_lik = -1254.56431, newton_decrement = 0.00186, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1254.56245, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6293001536824684


### c_index_mskcc

In [253]:
# 计算1年生存期分期的c_index_mskcc
df_C_index_mskcc = df_model_risk_before[['MSKCC','target_3Y_death','target_3Y_survival']]
df_C_index_mskcc = df_C_index_mskcc[df_C_index_mskcc.target_3Y_survival.notnull() & df_C_index_mskcc.MSKCC.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_mskcc, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_mskcc = cph.concordance_index_
print(c_index_mskcc)

Iteration 1: norm_delta = 0.40187, step_size = 0.9000, log_lik = -1273.65809, newton_decrement = 15.65977, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00933, step_size = 0.9000, log_lik = -1259.42443, newton_decrement = 0.01060, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00095, step_size = 0.9000, log_lik = -1259.41393, newton_decrement = 0.00011, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1259.41382, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6093805414351579


### c_index_blum

In [254]:
# 计算1年生存期分期的c_index_blumgart
df_C_index_blumgart = df_model_risk_before[['Blumgart_T','target_3Y_death','target_3Y_survival']]
df_C_index_blumgart = df_C_index_blumgart[df_C_index_blumgart.target_3Y_survival.notnull() & df_C_index_blumgart.Blumgart_T.notnull()]
cph = CoxPHFitter()
cph.fit(df_C_index_blumgart, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
c_index_blumgart = cph.concordance_index_
print(c_index_blumgart)

Iteration 1: norm_delta = 0.40676, step_size = 0.9000, log_lik = -1273.65809, newton_decrement = 16.20180, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00685, step_size = 0.9000, log_lik = -1258.85878, newton_decrement = 0.00567, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00069, step_size = 0.9000, log_lik = -1258.85316, newton_decrement = 0.00006, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1258.85310, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.
0.6107104858730347


In [255]:
df_model_risk_before.to_excel(project_path+'/data/reimplementation/df_8.2.4_分期c_index.xlsx')

## 计算BS

In [256]:
df_model_risk_before=pd.read_excel(project_path+'/data/reimplementation/df_8.1_术后分期.xlsx')
if 'Unnamed: 0' in df_model_risk_before.columns:
    df_model_risk_before = df_model_risk_before.drop(['Unnamed: 0'], axis=1)

In [257]:
# 术前预后1年生存期BS，生成布尔型True、False
df_model_bs_3Y=df_model_risk_before[df_model_risk_before.target_3Y_survival.notnull() & 
                                    df_model_risk_before.MSKCC.notnull()]
df_model_bs_3Y['target_death_binary']=df_model_bs_3Y.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_bs_3Y.target_death_binary,df_model_bs_3Y.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

### bs_us

In [258]:
# 术前1年BS_us
bs_x_us = df_model_bs_3Y[['staging']]
est_us = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_us, bs_y)
survs_us = est_us.predict_survival_function(bs_x_us)
preds_us = [fn(10) for fn in survs_us]
times, bs_3Y_us = brier_score(bs_y, bs_y, preds_us, 10)
print(bs_3Y_us)

NameError: name 'CoxPHSurvivalAnalysis' is not defined

### bs_AJCC

In [ ]:
# 术前1年BS_ajcc
bs_x_ajcc=df_model_bs_3Y[['AJCC_8']]
est_ajcc = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ajcc, bs_y)
survs_ajcc = est_ajcc.predict_survival_function(bs_x_ajcc)
preds_ajcc = [fn(10) for fn in survs_ajcc]
times, bs_3Y_ajcc = brier_score(bs_y, bs_y, preds_ajcc, 10)
print(bs_3Y_ajcc)

### bs_gazz

In [ ]:
# 术前1年BS_gazz
bs_x_gazz=df_model_bs_3Y[['Gazzaniga_T']]
est_gazz = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_gazz, bs_y)
survs_gazz = est_gazz.predict_survival_function(bs_x_gazz)
preds_gazz = [fn(10) for fn in survs_gazz]
times, bs_3Y_gazz = brier_score(bs_y, bs_y, preds_gazz, 10)
print(bs_3Y_gazz)

### bs_mskcc

In [ ]:
# 术前1年BS_mskcc
bs_x_mskcc=df_model_bs_3Y[['MSKCC']]
est_mskcc = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_mskcc, bs_y)
survs_mskcc = est_mskcc.predict_survival_function(bs_x_mskcc)
preds_mskcc = [fn(10) for fn in survs_mskcc]
times, bs_3Y_mskcc = brier_score(bs_y, bs_y, preds_mskcc, 10)
print(bs_3Y_mskcc)

### bs_blum

In [ ]:
# 术前1年BS_blumgart
bs_x_blumgart=df_model_bs_3Y[['Blumgart_T']]
est_blumgart = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_blumgart, bs_y)
survs_blumgart = est_blumgart.predict_survival_function(bs_x_blumgart)
preds_blumgart = [fn(10) for fn in survs_blumgart]
times, bs_3Y_blumgart = brier_score(bs_y, bs_y, preds_blumgart, 10)
print(bs_3Y_blumgart)